In [ ]:
jamil zakirov
jah zak
bon
jamil.zakir0v

# Imports

In [5]:
import os
import sys
import random 
import functools

import cv2
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt 
import albumentations as albu
import albumentations.pytorch as albu_pt
import photosynthesis_metrics as pm

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
%load_ext watermark
%watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

# %load_ext autoreload
# %autoreload 2

# Nice plot formating
%matplotlib inline

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Mon Jun 08 2020 

CPython 3.6.9
IPython 7.8.0

numpy 1.17.0
torch 1.5.0
albumentations 0.4.5
photosynthesis_metrics 0.3.0

compiler   : GCC 8.4.0
system     : Linux
release    : 4.15.0-99-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit


# Download datasets

In [ ]:
# mnist = torchvision.datasets.MNIST("../datasets", train=True, download=True)
# cifar10 = torchvision.datasets.CIFAR10("../datasets", download=True)
# cifar100 = torchvision.datasets.CIFAR100("../datasets", download=True)
# fashin_mnist = torchvision.datasets.FashionMNIST("datasets", download=True)

In [ ]:
# !wget -P datasets/ http://cs231n.stanford.edu/tiny-imagenet-200.zip
# !wget -P datasets/ http://vllab.ucmerced.edu/wlai24/LapSRN/results/SR_testing_datasets.zip
# !wget -P datasets/ http://www.cs.columbia.edu/CAVE/databases/SLAM_coil-20_coil-100/coil-100/coil-100.zip
# !wget -P datasets/ http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip
# !wget -P datasets/ http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
# !wget -P datasets/ http://www.ponomarenko.info/tid2013/tid2013.rar

# Test datasets

## TID2013

In [11]:
from src.data.datasets import TID2013

transform = albu.Compose([
#     albu.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), # to [-1, 1]
    albu.Normalize(mean=[0., 0., 0.], std=[1., 1., 1.]), # to [0, 1]
    albu_pt.ToTensorV2(),
])

dataset = TID2013(transform=transform)
distorted_images, reference_images, scores = dataset[0]

NameError: name 'albu_pt' is not defined

In [8]:
from src.augmentations import get_aug
from src.utils import walk_files
from src.datasets import *
from torch.utils.data import Dataset

In [9]:
TASK = "denoise"
SIZE = 256

dataset_class = {
    "mnist": MNIST,
    "fashion_mnist": FashionMNIST, 
    "cifar10": CIFAR10,
    "cifar100": CIFAR100,
    "tinyimagenet": TinyImageNet,
    "div2k": DIV2K,
    "set5": Set5,
    "set14": Set14,
    "urba100": Urban100,
    "manga109": Manga109,
    "coil100": COIL100,
    "bsds100": BSDS100,
    "medicaldecathlon": MedicalDecathlon
}


dataset_names = [
    "div2k",
    "bsds100",
#     "set5"
]

datasets = []
for dataset_name in dataset_names:
    transform = get_aug(aug_type="medium", task=TASK, dataset=datasets, size=SIZE)
    datasets.append(dataset_class[dataset_name](train=True, transform=transform))
    

TypeError: unhashable type: 'list'

In [ ]:
for dataset in datasets:
    print(dataset)
    input, target = dataset[1]
    print("Input", input.shape, input.min(), input.max(), input.mean())
    print("Target", target.shape, target.min(), target.max(), target.mean())
    plt.figure(figsize=(10, 15))
    plt.imshow(torch.cat([input, target], dim=2).permute(1, 2, 0) * 0.5 + 0.5, )
    plt.show()
    

In [ ]:
# total_mean, total_var = [], []
# for i in range(len(div2k)):
#     image = div2k[i][1]
#     total_mean.append(image.mean(dim=[1,2]))
#     total_var.append(image.var(dim=[1,2]))
# print(torch.stack(total_mean).mean(dim=0))
# print(torch.stack(total_var).mean(dim=0))



In [ ]:
# Get dataset with no transform
# AUG = get_aug(aug_type='light', dataset="medicaldecathlon", task="denoise", size=256)

# medicaldecathlon = MedicalDecathlon(train=False, transform=AUG)
# image = medicaldecathlon[135][0]
# image.dtype

In [ ]:
# Get dataset with no transform
# AUG = get_aug(aug_type='light', dataset="medicaldecathlon", task="deblur", size=256)

# medicaldecathlon = MedicalDecathlon(train=False, transform=AUG)
# image = medicaldecathlon[135][0]
# image.dtype

In [ ]:
print(image.min(), image.max(), image.mean())

In [ ]:
# idx = 356
print(image[..., 0].min(), image[..., 0].max(), image[...,0].mean())
# plt.hist()
# print(image.min(), image.max(), image.mean())
# grey_image = image[..., 0]
augmented = AUG(image=image, mask=image)
input, target = augmented["image"], augmented["mask"]

## Get gaussian
# random_state = np.random.RandomState(random.randint(0, 2 ** 32 - 1))
# gauss = random_state.normal(0, 0.1, input.shape)
# input = input + gauss
# gauss

# print("Input", input.shape, input.min(), input.max())
# print("Target", target.shape, target.min(), target.max())

# augmented = NORM_TO_TENSOR(image=input, mask=target)
# input, target = augmented["image"], augmented["mask"]

print("Input", input.shape, input.min(), input.max())
print("Target", target.shape, target.min(), target.max())
plt.figure(figsize=(10, 10))
plt.imshow(torch.cat([input[0], target[0]], dim=1)) #.permute(1, 2, 0))
# plt.imshow(torch.cat([input, target], dim=2).permute(1, 2, 0))

torch.sum(input - target)

In [ ]:
torch.mean(input - target)

In [ ]:
plt.imshow((medicaldecathlon[idx][0][..., 0]))

In [ ]:
a = np.random.rand(256, 256)
a = a[:,:,np.newaxis].repeat(3, axis=2)
# a.repeat(3, axis=0).shape

In [ ]:
plt.imshow(torch.cat([input, target], dim=2).permute(1, 2, 0)[:, :, [2, 1, 0]])

## Test dataloaders

In [ ]:
from src.datasets import get_dataloader

In [ ]:
datasets = ["div2k", "bsds100", "set5"]
transform = get_aug(aug_type="val", task="deblur", size=128)

for dataset in datasets:
#     train_loader = get_dataloader(dataset, transform, train=True, batch_size=64)
    
#     for batch in train_loader:
#         input, output = batch
#         print(input.shape, output.shape)
        
    
    val_loader = get_dataloader(dataset, transform, train=False)
    for batch in val_loader:
        input, output = batch
        print(input.shape, output.shape)
    
    
    for batch in train_loader:
        break

In [ ]:
dataset = BSDS100("datasets/BSDS100", train=True, transform=transform)
dataset[0][0].size()

In [ ]:
import photosynthesis_metrics as pm
image_metrics = ["kid", {}, "ssim", {}, ]

METRIC_FROM_NAME = {
    "ssim" : pm.SSIMLoss,
    "ms-ssim" : pm.MultiScaleSSIMLoss,
    "msid" : pm.MSID,
    "fid" : pm.FID,
    "kid" : pm.KID,
#     "content" : ContentLoss,
#     "style" : StyleLoss,
    "tv" : pm.TVLoss,
}


# for metric in image_metrics:
image_metrics = [METRIC_FROM_NAME[metric](**kwargs) for metric, kwargs in zip(image_metrics[::2], image_metrics[1::2])]
# list(zip(image_metrics[::2], image_metrics[1::2]))
image_metrics

In [ ]:
from collections import OrderedDict
output = OrderedDict({
    'loss': 1,
#     'mse': mse,
#     'psnr': psnr,
#     'ssim': ssim_score,
#     # 'val_ms_ssim': ms_ssim_score,
#     'input_features': input_features,
#     'target_features': target_features
})
output["test"] = 3
output["foo"] = "bar"

In [ ]:
images = torch.rand((512, 2, 32, 32))
target = torch.rand((512, 2, 64, 64))
print(f"Before interpolation: images {images.shape}")
images = F.interpolate(images, size=target.shape[-2:], mode="bilinear")
print(f"After interpolation: images {images.shape}")

In [ ]:
# from functools import b
# 
iter()

In [ ]:
a = ["foo", "bar"]
if "foo" in a:
    print(a.index("foo"))
# a.append("loss")
# print(a)
# a.pop()
# a
# for i in "etc", a:
#     print(i)

In [ ]:
features = make_layers(
    [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"], 
    batch_norm=True
)
vgg_16 = VGG(features)

In [ ]:
vgg_16

In [ ]:
x = torch.rand(10, 3, 224, 224)
layers = None
if layers is None:
    layers = ["0", "5", "10", "19", "28"]

features = []
for name, module in vgg_16.features._modules.items():
    x = module(x)
    if name in layers:
        features.append(x)


In [ ]:
features

In [ ]:
# from albumentations import ImageOnlyTransform
albu.ImageCompression()

In [ ]:
target.permute(2, 0, 1).shape, input.shape

In [ ]:
plt.imshow(target.permute(2, 0, 1).transpose(0, 2).transpose(0, 1))

In [ ]:
dataset = TinyImageNetDataset(train=True, transform=None, target_transform=None)
input, target = dataset[6]
# input.shape
# plt.imshow(input / 255.)
(input / 255.).min(), (input / 255.).max()

In [ ]:
input.transpose(0, 2).transpose(0, 1).shape

In [ ]:
# dataset[0][0].shape

In [ ]:
# from src.augmentations import get_aug
# from src.datasets import MNIST, CIFAR10, CIFAR100
from ..src.datasets
# from src.datasets import get_dataloader

In [ ]:
transform = get_aug(aug_type="light", size=32)
target_transform = get_aug(aug_type="val", size=32)
# target_transform = None

In [ ]:
transform, target_transform

In [ ]:
trainset = CIFAR10(root='../datasets', train=True, transform=transform, target_transform=target_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# valset = CIFAR10(root='../datasets', train=False, transform=transform, target_transform=target_transform)
# valloader = torch.utils.data.DataLoader(valset, batch_size=16,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
image, target = trainset[0]
print(image.shape, target.shape)
print(image.max(), image.min())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
input, target = dataiter.next()

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

model.fc = Identity()
model.to("cpu")

In [ ]:
mock = torch.rand((3, 3, 32, 32))

In [ ]:
result = model(mock)

In [ ]:
all_input_features = [result.detach() for _ in range(4)]
print(len(all_input_features))

In [ ]:
input_features = torch.cat(all_input_features, dim=0)
input_features.shape

In [ ]:
input.shape, target.shape

In [ ]:
import h5py

In [ ]:
datapath = "datasets/decathlon/colon.h5"
with h5py.File(datapath, "r") as f:
    for key in f.keys():
        print(key)
#     data_val = f['imgs_validation'][::10]
#     data_test = f['imgs_testing'][::10]
#     data = np.concatenate((data_val, data_test))
#     print(data.shape)
#     print(len(data))
    

# hf = 
# data_numpy = np.zeros(hf['imgs_train'].shape, dtype=numpy_type)
# # hf['dataset_name'].read_direct(n1)
# # hf.close()

In [ ]:
data = hf['imgs_train'][5000]
mask = hf['msks_train'][5000]
plt.figure(figsize=(10, 10))
plt.imshow(data.squeeze(), cmap='gray')
# plt.imshow(mask.squeeze(), alpha=0.1)
# data.max()


In [ ]:
# data.max()


In [ ]:
import torch
from torchvision import transforms
from torchvision.models import inception_v3                                                                                                                 
from torchvision.datasets import CIFAR10

import photosynthesis_metrics as pm    
from photosynthesis_metrics import IS
from tqdm.notebook import tqdm

In [ ]:
# mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]

aug = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

cifar10 = CIFAR10(root="/home/zakirov/repoz/metrics-comparison/datasets", train=True, transform=aug)
cifar10
loader = torch.utils.data.DataLoader(cifar10, batch_size=100, num_workers=10)

In [ ]:
# model = pm.feature_extractors.fid_inception.InceptionV3(
#     resize_input=True, 
#     normalize_input=True,
#     requires_grad=False,
#     use_fid_inception=False,
# )
model = inception_v3(pretrained=True, transform_input=False).cuda()
model.eval()

## Geometry score

In [ ]:
import photosynthesis_metrics as pm
import torch

In [ ]:
gs = pm.GS(num_iters=20)
f1 = torch.rand(1000, 20)
f2 = torch.rand(1000, 20)

In [ ]:
from scipy.spatial.distance import cdist

# VIF (debug and compare)

A computationally simpler, multi-scale pixel domain implementation whose performance is slightly worse than the Wavelet domain version presented in [2] can also be downloaded here . The pixel domain version also uses a scalar Random Field model for natural images, instead of a vector version in [2]. There are advantages to using the VIF in the pixel domain as well as some disadvantages. The principle advantage is computational simplicity. Secondly, the Wavelet transform used in [2] is a highly overcomplete decomposition that adds a lot of linear correlation between coefficients. While the wavelet decomposition allows scale-space-orientation analysis, this makes the assumptions of conditional independence in [2] weaker. Pixel domain methods avoid this weakness of assumption, but do not allow orientation analysis. 

# Read 2 images used in acticle and compute score for them

In [ ]:
transform = get_aug(aug_type='light', task='deblur', size=64)

loader = get_dataloader(
    datasets=['tinyimagenet'],
    train=True,
    transform=transform,
    batch_size=4
)

for batch in loader:
    break
    
lr = batch[0][1]
hr = batch[1][1]
print(lr.min(), lr.max())
plt.imshow((lr.permute(1,2,0) + 1) * 0.5)

# Geometry score

# Gradient Strength Error 

# Peceptual loss, Style loss, LPIPS?

In [ ]:
"""
Implementation of VGG16 loss, originaly used for style transfer and usefull in many other task (including GAN training)
It's work in progress, no guarantees that code will work
"""
import collections


import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss 
from torchvision.models import vgg16



def listify(p):
    if p is None:
        p = []
    elif not isinstance(p, collections.Iterable):
        p = [p]
    return p

class PSNR(_Loss):
    def forward(self, prediction, target):
        mse = torch.mean((prediction - target) ** 2)
        psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
        return psnr



class ContentLoss(_Loss):
    """
    Creates content loss for neural style transfer.
    Uses pretrained VGG16 model from torchvision by default
    layers: list of VGG layers used to evaluate content loss
    criterion: str in ['mse', 'mae'], reduction method
    """

    def __init__(
        self,
        layers=["21"],
        weights=1,
        loss="mse",
        device="cuda",
        **args,
    ):
        super().__init__()
        self.model = vgg16(pretrained=True, **args)
        self.model.eval().to(device)
        self.layers = listify(layers)
        self.weights = listify(weights)

        if loss == "mse":
            self.criterion = nn.MSELoss()
        elif loss == "mae":
            self.criterion = nn.L1Loss()
        else:
            raise KeyError

    def forward(self, input, content):
        """
        Measure distance between feature representations of input and content images
        """
        input_features = torch.stack(self.get_features(input))
        content_features = torch.stack(self.get_features(content))
        loss = self.criterion(input_features, content_features)

        # Solve big memory consumption
        torch.cuda.empty_cache()
        return loss

    def get_features(self, x):
        """
        Extract feature maps from the intermediate layers.
        """
        if self.layers is None:
            self.layers = ["21"]

        features = []
        for name, module in self.model.features._modules.items():
            x = module(x)
            if name in self.layers:
                features.append(x)
        # print(len(features))
        return features


class StyleLoss(_Loss):
    """
    Class for creating style loss for neural style transfer
    model: str in ['vgg16_bn']
    """

    def __init__(
        self,
        layers=["0", "5", "10", "19", "28"],
        weights=[0.75, 0.5, 0.2, 0.2, 0.2],
        loss="mse",
        device="cuda",
        **args,
    ):
        super().__init__() 
        self.model = vgg16(pretrained=True, **args)
        self.model.eval().to(device)

        self.layers = listify(layers)
        self.weights = listify(weights)

        if loss == "mse":
            self.criterion = nn.MSELoss()
        elif loss == "mae":
            self.criterion = nn.L1Loss()
        else:
            raise KeyError

    def forward(self, input, style):
        """
        Measure distance between feature representations of input and content images
        """
        input_features = self.get_features(input)
        style_features = self.get_features(style)
        # print(style_features[0].size(), len(style_features))

        input_gram = [self.gram_matrix(x) for x in input_features]
        style_gram = [self.gram_matrix(x) for x in style_features]

        loss = [
            self.criterion(torch.stack(i_g), torch.stack(s_g)) for i_g, s_g in zip(input_gram, style_gram)
        ]
        return torch.mean(torch.tensor(loss))

    def get_features(self, x):
        """
        Extract feature maps from the intermediate layers.
        """
        if self.layers is None:
            self.layers = ["0", "5", "10", "19", "28"]

        features = []
        for name, module in self.model.features._modules.items():
            x = module(x)
            if name in self.layers:
                features.append(x)
        return features

    def gram_matrix(self, input):
        """
        Compute Gram matrix for each image in batch
        input: Tensor of shape BxCxHxW
            B: batch size
            C: channels size
            H&W: spatial size
        """

        B, C, H, W = input.size()
        gram = []
        for i in range(B):
            x = input[i].view(C, H * W)
            gram.append(torch.mm(x, x.t()))
        return gram